In [2]:
%pip install gql 

ERROR: Exception:
Traceback (most recent call last):
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_internal/commands/install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_internal/metadata/pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "/Users/guy/opt/anaconda3/lib/python3.8/site-packages/pip/_vendor/packaging/version.py", line 56, in parse
    return Version

In [42]:
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from graphql import build_ast_schema, parse, print_schema



In [43]:
from dotenv import load_dotenv
import os
from typing import Optional, Dict, Any


load_dotenv()  # Load the .env file

GRAPHQL_API_KEY = os.environ.get("GRAPHQL_API_KEY")
GRAPHQL_ENDPOINT = os.environ.get("GRAPHQL_ENDPOINT")

class GraphQLClient:
    def __init__(self, endpoint: str):
        self.transport = AIOHTTPTransport(
            url=endpoint,
            headers={
                "x-api-key": GRAPHQL_API_KEY,
                'Content-Type': 'application/json',
            },
        )
        self.client = Client(
            transport=self.transport, 
            fetch_schema_from_transport=True
        )
        
    async def get_schema(self) -> str:
        """Fetch and return the GraphQL schema as a string"""
        async with self.client as session:
            schema = session.client.schema
            return print_schema(schema)
        # async with self.client as session:
        #     response = await session.execute(gql("query { __schema { types { name } } }"))
        #     schema = response['__schema']['types']
        #     return ' '.join([t['name'] for t in schema])
            
    async def execute_query(self, query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """Execute a GraphQL query and return the results"""
        async with self.client as session:
            result = await session.execute(gql(query), variable_values=variables)
            return result

client = GraphQLClient(GRAPHQL_ENDPOINT)

In [44]:


client

In [45]:
schema = await client.get_schema()


In [46]:
schema

'"""This directive allows results to be deferred during execution"""\ndirective @defer on FIELD\n\n"""Directs the schema to enforce authorization on a field"""\ndirective @aws_auth(\n  """List of cognito user pool groups which have access on this field"""\n  cognito_groups: [String]\n) on FIELD_DEFINITION\n\n"""\nTells the service which subscriptions will be published to when this mutation is called. This directive is deprecated use @aws_susbscribe directive instead.\n"""\ndirective @aws_publish(\n  """\n  List of subscriptions which will be published to when this mutation is called.\n  """\n  subscriptions: [String]\n) on FIELD_DEFINITION\n\n"""\nTells the service this field/object has access authorized by a Lambda Authorizer.\n"""\ndirective @aws_lambda on OBJECT | FIELD_DEFINITION\n\n"""\nTells the service this field/object has access authorized by an OIDC token.\n"""\ndirective @aws_oidc on OBJECT | FIELD_DEFINITION\n\n"""\nTells the service this field/object has access authorized 

In [36]:
query = """
query {
  organization {
    node_id
    name
  }
}
"""
vars_dict = None

In [39]:
result = await client.execute_query(query, vars_dict)

In [40]:
result['organization']

{'node_id': 'AQEtg9I7HiFD34gYqa5U5JsrOfglIOZlQ_C-0OjogD5gnA',
 'name': 'RentalX'}

In [41]:
from rich.console import Console
console = Console()
console.print(result)

{'organization': {'node_id': 'AQEtg9I7HiFD34gYqa5U5JsrOfglIOZlQ_C-0OjogD5gnA', 'name': 'RentalX'}}

In [47]:
async def generate_query(description: str) -> str:
    """Generate a GraphQL query based on the schema and user description"""
    try:
        schema = await client.get_schema()
        
        # Provide context about the schema and request to help generate the query
        prompt = f"""Given this GraphQL schema:

{schema}

Generate a GraphQL query that: {description}

Return only the GraphQL query without any explanation."""
        
        return prompt
    except Exception as e:
        return f"Error generating query: {str(e)}"

In [48]:
generated_query = await generate_query("name of organization")

In [49]:
generated_query

'Given this GraphQL schema:\n\n"""This directive allows results to be deferred during execution"""\ndirective @defer on FIELD\n\n"""Directs the schema to enforce authorization on a field"""\ndirective @aws_auth(\n  """List of cognito user pool groups which have access on this field"""\n  cognito_groups: [String]\n) on FIELD_DEFINITION\n\n"""\nTells the service which subscriptions will be published to when this mutation is called. This directive is deprecated use @aws_susbscribe directive instead.\n"""\ndirective @aws_publish(\n  """\n  List of subscriptions which will be published to when this mutation is called.\n  """\n  subscriptions: [String]\n) on FIELD_DEFINITION\n\n"""\nTells the service this field/object has access authorized by a Lambda Authorizer.\n"""\ndirective @aws_lambda on OBJECT | FIELD_DEFINITION\n\n"""\nTells the service this field/object has access authorized by an OIDC token.\n"""\ndirective @aws_oidc on OBJECT | FIELD_DEFINITION\n\n"""\nTells the service this field